<a href="https://colab.research.google.com/github/amitroyal8755/NLP/blob/main/imdb_dataset_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices("GPU") else "NOT AVAILABLE")

In [ ]:
# Split the training set into 60% and 40% to end up with 15,000 examples
# for training, 10,000 examples for validation and 25,000 examples for testing.


In [ ]:
imdb= tfds.load(name="imdb_reviews",
    as_supervised=True)

In [ ]:
train_data,test_data=imdb['train'],imdb['test']

In [ ]:
x_train_1=[]
y_train_1=[]
x_test_1=[]
y_test_1=[]
for i,k in train_data:
  x_train_1.append(str(i.numpy()))
  y_train_1.append(k.numpy())
for i,k in test_data:
  x_test_1.append(str(i.numpy()))
  y_test_1.append(k.numpy())
y_train=np.array(y_train_1)
y_test=np.array(y_test_1)

In [ ]:
from tensorflow import keras 
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
len(x_train_1)

In [ ]:
vocub_size=25000
oov='<oov>'
max_length=120
trunk_type='post'


In [ ]:
t1=Tokenizer(num_words=vocub_size,oov_token=oov)

In [ ]:
t1.fit_on_texts(x_train_1)

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
word_index=t1.word_index

In [ ]:
seq=t1.texts_to_sequences(x_train_1)

In [ ]:
x_train=pad_sequences(seq,maxlen=max_length,truncating=trunk_type)

In [ ]:
seq_1=t1.texts_to_sequences(x_test_1)

In [ ]:
x_test=pad_sequences(seq_1,maxlen=max_length,truncating=trunk_type)

In [ ]:
inputs = keras.Input(shape=(max_length))
l1=keras.layers.Embedding(input_dim=vocub_size,output_dim=16,)(inputs)
l2=keras.layers.Flatten()(l1)
l3=keras.layers.Dense(6, activation='relu')(l2)
outputs=keras.layers.Dense(1, activation='sigmoid')(l3)
model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
history = model.fit(x_train,y_train,
                    epochs=10,
                    validation_data=(x_train,y_train),
                    verbose=1)

In [ ]:
history_dict = history.history
history_dict.keys()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']
loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1, len(acc) + 1)

# "bo" is for "blue dot"
plt.plot(epochs, loss, label='Training loss')
# b is for "solid blue line"
plt.plot(epochs, val_loss, 'b',color='orange' ,label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
plt.plot(epochs, acc, color='orange', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')

plt.show()